<a href="https://colab.research.google.com/github/dbetteb/early-ML/blob/master/03_LINEAR_MODELS/Bayesian_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression Linéaire Bayésienne

Ce notebook a pour ambition de présenter et d'illustrer les concepts fondamentaux du Machine Learning bayésien à commencer par la régression linéaire bayésienne.

## Fondamentaux théoriques : le cas Gaussien

La ML bayésien a pour but principal d'inférer des informations _sur_ les paramètres d'un modèle _à partir_ des observations. En résumé, l'objectif du ML bayésien est souvent d'estimer :
$$ \mathbb{P}(w|\mathcal{D}) $$

c'est-à-dire la distribution de probabilité des paramètres $w$ d'un modèle de ML en ayant observé les données $\mathcal{D}=(x_i, y_i)_{i=1\ldots N}$

Pour ramener cette estimation dans la perspective bayésienne, observons que cette probabilité est bien la probabilité _inverse_ : la probabilité des causes (les paramètres du modèles) conditionnée aux _observations_. En appliquant classiquement la formule de Bayes on a :

$$ \mathbb{P}(w|\mathcal{D}) = \frac{\mathbb{P}(\mathcal{D}|w)\mathbb{P}(w)}{\mathbb{P}(\mathcal{D})}$$



Dans le cas du modèle linéaire Gaussien, on écrit :

$$\mathbb{P}(\mathcal{D}|w) = \mathbb{P}(y|x,w) = \mathcal{N}(y|w_0 + w^Tx, \sigma^2)$$

autrement dit, on modélise les données comme étant les réalisations d'une loi Gaussienne centrée autour d'une combinaison linéaire des entrées.

On va imposer un a priori Gaussien sur les paramètres $w$ de notre modèle 

$$\mathbb{P}(w) = \mathcal{N}(w|\mu_0^w, V_0^w)$$

autrement dit, on met comme a priori que $w$ soit une loi Gaussienne multivariée. Afin d'illustrer cette notion on va répresenter ce que cet a priori signifie dans le cas d'un modèle simple de la forme $ax+b$ 

In [ ]:
import numpy as np

mu0 = np.array([0.,0.])
V0  = np.array([[1.,0.],[0.,1.0]])

On va désormais tirer quelques paramètres de cet loi et représenter les modèles linéaires résultats. Chaque tirage de la loi Gaussienne, représente un modèle possible de la forme $ax+b$.

Avant ça, on charge une fonction permettant d'afficher des graphiques `plotly`.

In [ ]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)
#for google colab
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

On fait quelques tirages et on affiche les modèles.

In [ ]:
configure_plotly_browser_state()
data = []
Nsamples=5
for i in range(0,Nsamples):
    a,b=np.random.multivariate_normal(mu0, V0)
    trace = Scatter(
        x=np.linspace(-1,1,50),
        y=a*np.linspace(-1,1,50)+b,
        mode='lines',
        name=f"Model with a : {a} and b : {b}",
    )
    data.append(trace)


fig = dict(data=data, layout={})
fig['layout']['xaxis'] = dict(title='Variable X')
fig['layout']['yaxis'] = dict(title='Variable Y')

iplot(fig)

Maintenant qu'on dispose :


*   d'un a priori $\mathbb{P}(w)$
*   d'un modèle de la génération des données direct $\mathbb{P}(\mathcal{D}|w)$

On peut appliquer le théorème de Bayes pour obtenir :

$$\mathbb{P}(w|\mathcal{D}=(x_i,y_i)) = \frac{\mathbb{P}(w)\mathbb{P}(\mathcal{D}|w)}{\mathbb{P}(\mathcal{D})}$$

comme souvent, la dénominateur ne peut être calculé simplement, par contre les termes du numérateur sont des lois Gaussiennes 

$$\mathbb{P}(w|\mathcal{D}=(x_i,y_i)) \sim \mathcal{N}(y|w_0 + w^Tx, \sigma^2) \mathcal{N}(w|\mu_0^w, V_0^w)$$




La loi de $w|\mathcal{D}=(x_i,y_i)_{i=1\ldots N})$ est une Gaussienne :

$$\mathbb{P}(w|\mathcal{D}=(x_i,y_i)) \sim \mathcal{N}(w | \mu_N^w, V_N^w)$$

où $\mu_N^w$ et $V_N^w$ sont mis à jour en fonction des données $(x_i, y_i)$ 

$$ V_N^w = \Big((V_0^w)^{-1}+\frac{1}{\sigma^2} X^T X \Big)^{-1} $$

et 

$$ \mu_N^w = V_N^w (V_0^w)^{-1} \mu_0^w + \frac{1}{\sigma^2}V_N^w X^Ty$$

où

$$
X = \begin{pmatrix}
1 & x_1^{(1)} & \ldots & x_1^{(d)}\\
\vdots & \vdots & \ldots & \\
1 & x_N^{(1)} & \ldots & x_N^{(d)}
\end{pmatrix}
$$
et 

$$
y = \begin{pmatrix}
y_1 \\
\vdots \\
y_N
\end{pmatrix}
$$

Par exemple, si on a trois observations $(0.2,0.5)$, $(0.3,0.55)$ et $(0.5,0.8)$ on a 

$$
X = \begin{pmatrix}
1 & 0.2\\
1 & 0.3\\
1 & 0.5
\end{pmatrix}
$$

et 

$$
y = \begin{pmatrix}
0.5\\
0.55\\
0.8
\end{pmatrix}
$$

In [ ]:
configure_plotly_browser_state()
data = []
trace = Scatter(
  x=np.array([0.2,0.3,0.5]),
  y=np.array([0.5,0.55,0.8]),
  mode='markers',
  name=f"Observations",
  )
data.append(trace)
fig = dict(data=data, layout={})
fig['layout']['xaxis'] = dict(title='Variable X')
fig['layout']['yaxis'] = dict(title='Variable Y')

iplot(fig)

On va appliquer les formules vues plus haut qui vont nous permettre d'échantilloner des paramètres $ax+b$ de modèles sur la distribution à posteriori. On commence par paramètrer (pour illustrer simplement) la matrice de covariance des paramètres à priori sous la forme $V_0^w = \tau^2 I$ et on suppose connu la variance $\sigma$. 

In [ ]:
sigma = 0.1
tau   = 1.

X = np.array([[1.,0.2],[1.,0.3],[1.,0.5]])
y = np.array([[0.5],[0.55],[0.8]])

V0winv = 1./tau**2*np.eye(2)
Vnw = sigma**2*np.linalg.inv(sigma**2*V0winv+X.transpose() @ X)
muNw = Vnw @ (V0winv @ np.array([[0],[0]]))+1/sigma**2*(Vnw @ X.transpose() @ y)

_NB_ Le calcul précédent cherche simplement à appliquer les formules vues plus haut et ne vise pas l'efficacité

On vérifie les valeurs de $\mu_n^w$ et $V_n^w$

In [ ]:
muNw

array([[0.32489937],
       [0.8720529 ]])

In [ ]:
Vnw

array([[ 0.02242668, -0.05750431],
       [-0.05750431,  0.17308798]])

In [ ]:
configure_plotly_browser_state()
sigma = 0.1
tau   = 1.

X = np.array([[1.,0.2],[1.,0.3],[1.,0.5]])
y = np.array([[0.5],[0.55],[0.8]])

V0winv = 1./tau**2*np.eye(2)
Vnw = sigma**2*np.linalg.inv(sigma**2*V0winv+X.transpose() @ X)
muNw = Vnw @ (V0winv @ np.array([[0],[0]]))+1/sigma**2*(Vnw @ X.transpose() @ y)

data = []
Nsamples=100
for i in range(0,Nsamples):
    b,a=np.random.multivariate_normal(muNw.flatten(), Vnw)
    trace = Scatter(
        x=np.linspace(-1,1,50),
        y=a*np.linspace(-1,1,50)+b,
        mode='lines',
        line=dict(width=2,color='red'),
        name=f"Model with a : {a} and b : {b}",
    )
    data.append(trace)

trace = Scatter(
  x=np.array([0.2,0.3,0.5]),
  y=np.array([0.5,0.55,0.8]),
  mode='markers',
  marker=dict(size=8),
  name=f"Observations",
  )
data.append(trace)
fig = dict(data=data, layout={})
fig['layout']['xaxis'] = dict(title='Variable X')
fig['layout']['yaxis'] = dict(title='Variable Y')

iplot(fig)

On va modifier le code précédent pour calculer un intervalle crédible de manière empirique (on verra par la suite que dans le cas Gaussien on peut obtenir cet intervalle crédible de manière analytique).

In [ ]:
configure_plotly_browser_state()
sigma = 0.1
tau   = 1.

X = np.array([[1.,0.2],[1.,0.3],[1.,0.5]])
y = np.array([[0.5],[0.55],[0.8]])

V0winv = 1./tau**2*np.eye(2)
Vnw = sigma**2*np.linalg.inv(sigma**2*V0winv+X.transpose() @ X)
muNw = Vnw @ (V0winv @ np.array([[0],[0]]))+1/sigma**2*(Vnw @ X.transpose() @ y)

data = []
Nsamples=100
x=np.linspace(-1,1,50)

samp = np.zeros((Nsamples, 50))
for i in range(0,Nsamples):
    b,a=np.random.multivariate_normal(muNw.flatten(), Vnw)
    samp[i,:]=a*x+b

In [ ]:
configure_plotly_browser_state()

sigma = 0.05
tau   = 10.

X = np.array([[1.,0.2],[1.,0.3],[1.,0.5]])
y = np.array([[0.5],[0.55],[0.8]])

V0winv = 1./tau**2*np.eye(2)
Vnw = sigma**2*np.linalg.inv(sigma**2*V0winv+X.transpose() @ X)
muNw = Vnw @ (V0winv @ np.array([[0],[0]]))+1/sigma**2*(Vnw @ X.transpose() @ y)
Nsamples=100
x=np.linspace(-1,1,50)
samp = np.zeros((Nsamples, 50))
for i in range(0,Nsamples):
    b,a=np.random.multivariate_normal(muNw.flatten(), Vnw)
    samp[i,:]=a*x+b


trace1 = Scatter(
  x=np.linspace(-1,1,50),
  y=np.quantile(samp,0.90,axis=0),
  mode='lines',
  line=dict(width=2,color='red'),
  name=f"0.75 quantile",
  )

trace2 = Scatter(
  x=np.linspace(-1,1,50),
  y=np.quantile(samp,0.10,axis=0),
  mode='lines',
  line=dict(width=2,color='red'),
  name=f"0.25 quantile",
  )

trace4 = Scatter(
  x=np.linspace(-1,1,50),
  y=np.quantile(samp,0.50,axis=0),
  mode='lines',
  line=dict(width=2,color='black'),
  name=f"mean",
  )

trace3 = Scatter(
  x=np.array([0.2,0.3,0.5]),
  y=np.array([0.5,0.55,0.8]),
  mode='markers',
  marker=dict(size=8),
  name=f"Observations",
  )

fig = dict(data=[trace1, trace2, trace3, trace4], layout={})
fig['layout']['xaxis'] = dict(title='Variable X')
fig['layout']['yaxis'] = dict(title='Variable Y')

iplot(fig)

## Exercice 1

Rejouer la précédente analyse avec différentes valeurs de $\tau$ et $\sigma$.

## Régression Linéaire Bayésienne approchée

Dans le cas général, on ne peut pas écrire analytiquement la forme de la distribution a posteriori et encore moins la prédictive postérieure. On a recours à des techniques d'inférence bayésienne approchée :


*   Monte-Carlo Markov Chain
*   Inférence variationnelle

